In [1]:
from lxml import etree
import lxml.html
import wikitextparser as wtp
import nltk.data
import re

In [3]:
tree = etree.parse('/home/achang/Downloads/simplewiki-20171103-pages-meta-current.xml')

In [4]:
namespaces = {
    'mediawiki': 'http://www.mediawiki.org/xml/export-0.10/'
}

In [88]:
index = 3159
page = tree.xpath(f'mediawiki:page[{index}]', namespaces = namespaces)[0]
redirect = tree.xpath(f'mediawiki:page[{index}]/mediawiki:redirect', namespaces = namespaces)
ns = tree.xpath(f'mediawiki:page[{index}]/mediawiki:ns', namespaces = namespaces)
title = tree.xpath(f'mediawiki:page[{index}]/mediawiki:title', namespaces = namespaces)[0].text
text = tree.xpath(f'mediawiki:page[{index}]/mediawiki:revision/mediawiki:text', namespaces = namespaces)[0].text
title, ns[0].text, redirect

('Devon', '0', [])

In [89]:
print(text)

[[File:EnglandDevon.png|right|framed|A [[map]] showing where Devon is in England (coloured [[red]])]]
'''Devon''' is a [[county]] in southwest [[England]]. Sometimes, it is called ''Devonshire'' although this is not its correct name. Devon is the second largest county in England, and has the longest road network of any county in England.

== History ==
After the last [[ice age]], Devon was one of the first places in England where people started to live. [[Archaeologist]]s have found many old places in Devon with [[wikt:ancient|ancient]] buildings. For example, many ruins of old buildings have been found in an area called "[[Dartmoor]]", which is now a [[National Park]].

Devon gets its name from the ''Dumnonii'', a name that the invading [[Roman Britain|Romans]] gave to the Celtic tribe in that area. The Romans invaded Devon about [[AD]] 50. The name ''Dumnonii'' means "a person who lives in a deep [[valley]]", and it comes from the [[hill]]s and valleys of the area. The Roman [[army]]

In [82]:
def parse(raw_text):
    keep_tags = {'i','p','u','b','tt'}
    wiki_text = wtp.parse(raw_text)
    regions = []
    strs = []
    links = []
    offset = 0
    
    for l in wiki_text.wikilinks:
        span = l.span
        discard = re.match(r'^[^:]+:', l.target) and not l.target.startswith('wikt:')
        target = l.target if not discard and not l.target.startswith('wikt:') else None
        text = l.text or l.target
        regions.append((span, discard, target, text))
        
    for l in wiki_text.external_links:
        span = l.span
        discard = False
        target = None
        text = l.text or l.url
        regions.append((span, discard, target, text))
        
    for t in wiki_text.templates:
        span = t.span
        discard = True
        target = None
        text = None
        regions.append((span, discard, target, text))
    
    for t in wiki_text.tags():
        span = t.span
        target = None
        try:
            discard = t.name not in keep_tags
            text = t.contents
        except TypeError:
            discard = True
            text = None
        regions.append((span, discard, target, text))
    
    for t in wiki_text.tables:
        span = t.span
        discard = True
        target = None
        text = None
        regions.append((span, discard, target, text))
    
    for l in wiki_text.lists():
        span = l.span
        discard = True
        target = None
        text = None
        regions.append((span, discard, target, text))
        
    for pf in wiki_text.parser_functions:
        span = pf.span
        discard = True
        target = None
        text = None
        regions.append((span, discard, target, text))
        
    for c in wiki_text.comments:
        span = c.span
        discard = True
        target = None
        text = None
        regions.append((span, discard, target, text))
        
    regions.sort()
    
    for (span, discard, target, text) in regions:
        if span[0] < offset:
            continue
        strs.append(wiki_text.string[offset:span[0]])
        if not discard:
            if target:
                strs.append('{{%d}}' % len(links))
                links.append((text, target))
            else:
                strs.append(text)
        offset = span[1]
    
    strs.append(wiki_text.string[offset:])
    
    text = ''.join(strs)
    
    text = re.sub(r"''+", '', text)
    text = re.sub(r"==+[^=\n]+==+", '', text)
    text = re.sub('&nbsp;', ' ', text)
    text = re.sub('\n\n+', '\n\n', text)
    text = text.strip()
    
    return text, links

In [90]:
parsed_text, links = parse(text)
print(parsed_text)
print(links)

Devon is a {{0}} in southwest {{1}}. Sometimes, it is called Devonshire although this is not its correct name. Devon is the second largest county in England, and has the longest road network of any county in England.

After the last {{2}}, Devon was one of the first places in England where people started to live. {{3}}s have found many old places in Devon with ancient buildings. For example, many ruins of old buildings have been found in an area called "{{4}}", which is now a {{5}}.

Devon gets its name from the Dumnonii, a name that the invading {{6}} gave to the Celtic tribe in that area. The Romans invaded Devon about {{7}} 50. The name Dumnonii means "a person who lives in a deep {{8}}", and it comes from the {{9}}s and valleys of the area. The Roman {{10}} stayed in Devon for about 25 years. Their {{11}} was in the city of {{12}}.

It was a long time before anyone else invaded Devon. {{13}} came to Devon in the {{14}}, and the King of {{15}} may have attacked in {{16}}. There was 

In [73]:
parsed_text[17:31]

'transliterated'

In [41]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [84]:
tokenizer.tokenize(parsed_text)

['(; also {{0}} Leo, Lev, Trotskii, Trotski, Trotskij and Trotzky; October 26 ({{1}}) = November 7 ({{2}}), {{3}} in {{4}} (nowadays {{5}}) - August 21, {{6}} in {{7}}, {{8}}) was the public name of Leib or Lev Davidovich Bronstein.',
 'He was a {{9}} {{10}} and political {{11}}.',
 'He was of {{12}}-Ukrainian descent.',
 'Trotsky was a {{13}} and a {{14}}.',
 'In 1905 there was a revolution in Russia.',
 'Trotsky helped organize a {{15}} of workers in {{16}} which was called the {{17}}.',
 'When the leader of the Soviet was arrested, Trotsky became leader of the Soviet in his place.',
 'Trotsky helped to expand on the idea of {{18}}, which is a {{19}} originally thought up by {{20}}.',
 'After the revolution Trotsky left Russia for his own safety.',
 'In 1917, he returned to Russia because of the {{21}}.',
 'He helped to bring about communism in Russia and defend it during the Russian Civil War.',
 'He was in charge of the army at this time.',
 'When the leader of the {{22}} (the part